**install all dependencies**

In [1]:
!pip install transformers datasets accelerate tqdm torch rouge-score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_c

loading datasets

In [4]:
from sklearn.utils import shuffle
from datasets import load_dataset

naits_lfqa = load_dataset('fahdsoliman/naits_lfqa_processed')
old_lfqa = load_dataset('fahdsoliman/lfqa_with_supports_subset')

HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://huggingface.co/api/datasets/fahdsoliman/naits_lfqa_processed (Request ID: Root=1-66a546e8-55557b365d0169c41ffe72d1;2bc80b0f-2f94-4df7-9a92-5043a384b254)

Internal Error - We're working hard to fix this as soon as possible!

In [ ]:
old_lfqa

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 3004
    })
    validation: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 286
    })
    test: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 108
    })
})

In [3]:
# loading LFQA model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
model_name = "vblagoje/bart_lfqa"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [4]:
def dataset_to_list(dataset):
    return [example for example in dataset]

In [6]:
naits_lfqa_list = dataset_to_list(naits_lfqa['train'])
naits_lfqa_val_list = dataset_to_list(naits_lfqa['validation'])
old_lfqa_list = dataset_to_list(old_lfqa['train'])
old_lfqa_val_list = dataset_to_list(old_lfqa['validation'])

NameError: name 'naits_lfqa' is not defined

In [ ]:
class MemoryBuffer:
    def __init__(self, capacity):
        self.buffer = []
        self.capacity = capacity
        self.current_index = 0

    def add(self, samples):
        self.buffer.extend(samples)
        if len(self.buffer) > self.capacity:
            self.buffer = self.buffer[:self.capacity]  # الحفاظ على الحد الأقصى للسعة فقط

    def sample(self, batch_size):
        if len(self.buffer) == 0:
            raise ValueError("The buffer is empty")

        start_index = self.current_index
        end_index = (self.current_index + batch_size) % len(self.buffer)

        if start_index < end_index:
            samples = self.buffer[start_index:end_index]
        else:
            samples = self.buffer[start_index:] + self.buffer[:end_index]

        self.current_index = end_index
        return samples

# Initialize the memory buffer
buffer_capacity = 1000  # You can adjust this
buffer_val_capacity = 100
train_memory_buffer = MemoryBuffer(buffer_capacity)
validate_memory_buffer = MemoryBuffer(buffer_capacity)
# Add samples from the original LFQA dataset
train_memory_buffer.add(old_lfqa_list)
validate_memory_buffer.add(old_lfqa_val_list)

In [ ]:
batch = 1
for i in range(50):
  print(train_memory_buffer.sample(batch_size=batch))

[{'question': 'do caterpillars turn into liquid and then reform into a butterfly in the cocoon? or do they keep some of the same parts?', 'context': '<p> to begin the pupal or chrysalis stage, the caterpillar attaches itself with a silk pad and hangs upside down on a leaf. about 24 hours later, its skin splits, exposing a dull, bronze-coloured case known as the pupa or chrysalis. the pupa hangs for about a week without movement. within the pupa, the caterpillar is turning into liquid and changing into a butterfly, a process called metamorphosis.\n <p> most moth caterpillars spin a cocoon made of silk within which they metamorphose into the pupal stage. most butterfly caterpillars, on the other hand, form an exposed pupa made from a hardened protein, also termed a chrysalis.\n <p> once the caterpillars hatch, they are fairly solitary. they eat leaves from the edge inwards. the caterpillars eat from several host plants. caterpillars begin as yellow with black stripes, but become blue gre

In [ ]:
import random
from datasets import Dataset

class CombinedDataset:
    def __init__(self, new_dataset, memory_buffer, buffer_ratio=0.5, batch_size=4):
        self.new_dataset = new_dataset
        self.memory_buffer = memory_buffer
        self.buffer_ratio = buffer_ratio
        self.batch_size = batch_size

        self.num_new_samples = len(self.new_dataset)
        self.num_old_samples = int(self.num_new_samples * (1 - self.buffer_ratio) / self.buffer_ratio)
        self.total_length = self.num_new_samples + self.num_old_samples

        self.num_batches = (self.total_length + 3) // self.batch_size
        self.num_new_per_batch = int(self.buffer_ratio * self.batch_size)
        self.num_old_per_batch = self.batch_size - self.num_new_per_batch

        new_samples = list(self.new_dataset)
        old_samples = random.sample(self.memory_buffer.buffer, self.num_batches * self.num_old_per_batch)

        self.all_samples = self.merge_samples(new_samples, old_samples)

    def merge_samples(self, new_samples, old_samples):
        merged_samples = []
        new_idx = 0
        old_idx = 0

        for _ in range(self.num_batches):
            for _ in range(self.num_new_per_batch):
                if new_idx < len(new_samples):
                    merged_samples.append(new_samples[new_idx])
                    new_idx += 1
            for _ in range(self.num_old_per_batch):
                if old_idx < len(old_samples):
                    merged_samples.append(old_samples[old_idx])
                    old_idx += 1

        return merged_samples

    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        return self.all_samples[idx]

    def get_all_samples(self):
        return Dataset.from_list(self.all_samples)


In [ ]:
import numpy as np
label_pad_token_id = -100
def tokenize_dataset(examples):
    inputs = ["question: {} context: {}".format(q, c) for q, c in zip(examples["question"], examples["context"])]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, max_length=1024, padding="max_length", truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=360, padding=True, truncation=True,
                            return_tensors="np")

    model_inputs["decoder_input_ids"] = labels["input_ids"][:, :-1].tolist()
    # replace pad_token_id with label_pad_token_id to avoid loss calculation on those tokens
    labels["input_ids"] = np.where(labels["input_ids"] == tokenizer.pad_token_id,
                                    label_pad_token_id, labels["input_ids"])

    model_inputs["labels"] = labels["input_ids"][:, 1:].tolist()
    return model_inputs

In [ ]:
from datasets import DatasetDict

combined_train_dataset = CombinedDataset(naits_lfqa_list, train_memory_buffer, buffer_ratio=0.75)
combined_val_dataset = CombinedDataset(naits_lfqa_val_list, validate_memory_buffer, buffer_ratio=0.75)

# Get all samples
all_train_samples = combined_train_dataset.get_all_samples()
all_val_samples = combined_val_dataset.get_all_samples()

old_combined_lfqa = DatasetDict({'train': all_train_samples, 'validation': all_val_samples})

In [ ]:
old_combined_lfqa

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 34
    })
})

In [ ]:
for i in range(60):
  print(naits_lfqa_list[i])

In [ ]:
for i in range(300):
    print(old_combined_lfqa['train'][i])

{'answer': 'If you do not pay the extension fee for your shared hosting, it will be paused for three months. During this period, you will not have access to the hosting, and will not be able to retrieve any data. If the fees due are not paid within these three months, the hosting will be permanently cancelled, and you will not have the right to request any backup copy of the hosting data. According to Article (41), the procedure begins with temporarily stopping the service for a period of up to three months, and if the fees are not paid, the hosting is permanently canceled without the possibility of recovering data. If the hosting is affiliated with a public sector entity, the procedures are slightly different, as the authority sends a financial claim a month before the due date to remind the entity of the necessity of renewal.', 'question': 'i have shared hosting that i reserved a year ago, but i received an email saying that this hosting has been suspended due to non-payment of the e

IndexError: Invalid key: 80 is out of bounds for size 80

In [ ]:
all_train_samples

Dataset({
    features: ['answer', 'question', 'context'],
    num_rows: 120
})

In [ ]:
tokenized_datasets = {}
for split in ["train", "validation"]:
        tokenized_datasets[split] = old_combined_lfqa[split].map(tokenize_dataset,
                                                    batched=True,
                                                    remove_columns=old_combined_lfqa[split].column_names,
                                                    desc="Preparing dataset for tokenization")

Preparing dataset for tokenization:   0%|          | 0/80 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Preparing dataset for tokenization:   0%|          | 0/34 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets['validation'][0])

{'input_ids': [0, 40018, 35, 141, 64, 939, 1306, 127, 414, 40, 1091, 12143, 8, 2823, 77, 4098, 19, 5, 2033, 335, 573, 518, 138, 116, 5377, 35, 28696, 642, 15698, 24, 16, 888, 5049, 8, 2329, 16375, 7, 6, 8, 1200, 9, 786, 12, 33182, 32, 2006, 4, 2623, 708, 7, 432, 19, 1923, 4495, 35, 2623, 10, 278, 9, 20552, 5588, 7, 3616, 8, 1100, 5, 24147, 9, 10, 573, 6999, 4, 810, 4990, 35, 5, 6818, 24126, 7089, 14, 2386, 9397, 5, 2476, 14, 189, 28, 4924, 7, 5, 467, 8, 5, 3038, 14, 115, 898, 31, 106, 8, 15670, 5, 2139, 6196, 19, 5, 4374, 9, 1903, 1052, 185, 5, 3901, 2390, 7, 432, 19, 167, 4, 12114, 4, 573, 15661, 4990, 35, 5, 609, 9, 6062, 8, 3219, 70, 801, 10778, 8, 18659, 11, 41, 335, 467, 13, 5, 3508, 9, 1380, 4945, 106, 4, 28696, 642, 15698, 11, 41, 335, 467, 13, 5, 3508, 9, 20257, 6, 1966, 8, 10437, 4, 19157, 3044, 35, 41, 3319, 544, 14, 1171, 10, 35333, 2088, 573, 24333, 544, 6, 11, 1285, 7, 3044, 5, 19157, 9, 5, 2111, 18, 1743, 11, 1319, 14, 34233, 10, 588, 908, 11, 13141, 19, 5, 2111, 8, 109, 

In [ ]:
from datasets import concatenate_datasets

preprocessed_dataset = concatenate_datasets([naits_lfqa['test'], old_lfqa['test']])
preprocessed_dataset2 = old_lfqa['test']


In [ ]:
# evaluate function

from datasets import load_metric, load_dataset
from tqdm.auto import tqdm
import torch


def evaluate_model(preprocessed_dataset):
    metric_rouge = load_metric("rouge", trust_remote_code=True)
    metric_bleu = load_metric("sacrebleu", trust_remote_code=True)
    all_predictions = []
    all_labels = []
    model.eval()

    for example in tqdm(preprocessed_dataset):
        # إعداد المدخلات
        inputs = tokenizer("question: {} context: {}".format(example["question"], example["context"]), truncation=True, padding=True, return_tensors="pt")
        # print(inputs)
        reference = example["answer"]

        # توليد التنبؤات
        with torch.no_grad():
            outputs = model.generate(input_ids=inputs["input_ids"].to(device),
                                    attention_mask=inputs["attention_mask"].to(device),
                                    min_length=64,
                                    max_length=256,
                                    do_sample=False,
                                    early_stopping=True,
                                    num_beams=8,
                                    temperature=1.0,
                                    top_k=None,
                                    top_p=None,
                                    eos_token_id=tokenizer.eos_token_id,
                                    no_repeat_ngram_size=3,
                                    num_return_sequences=1)

        prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        # تجميع التنبؤات والمراجع
        all_predictions.append(prediction)
        all_labels.append(reference)

    # حساب ROUGE
    rouge_result = metric_rouge.compute(predictions=all_predictions, references=all_labels, use_stemmer=True)
    rouge_scores = {key: value.mid.fmeasure * 100 for key, value in rouge_result.items()}

    # حساب BLEU
    bleu_result = metric_bleu.compute(predictions=all_predictions, references=[[ref] for ref in all_labels])
    bleu_score = bleu_result["score"]

    print("ROUGE Scores:", rouge_scores)
    print("BLEU Score:", bleu_score)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from torch.utils.data import DataLoader
from accelerate import Accelerator
from accelerate.utils import set_seed
from transformers import DataCollatorWithPadding, get_scheduler
import torch

accelerator = Accelerator()

# إنشاء DataCollator لتجميع البيانات
data_collator = DataCollatorWithPadding(tokenizer)

# إعداد DataLoader للتدريب والاختبار
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=False, batch_size=6, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["validation"], shuffle=False, batch_size=6, collate_fn=data_collator)

# إعداد النموذج والتدريب باستخدام Accelerator
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, torch.optim.AdamW(model.parameters(), lr=1e-6, weight_decay=0.0), train_dataloader, eval_dataloader
)

# جدولة التعلم
num_training_steps = len(train_dataloader) * 3  # عدد العصور
num_eval_steps = len(eval_dataloader) * 3
print(num_training_steps)
print(num_eval_steps)
num_warmup_steps = int(0.05 * num_training_steps)
print(num_warmup_steps)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps
)


42
18
2


In [ ]:
from huggingface_hub import Repository
from tqdm.auto import tqdm

# إعداد المستودع لرفع النموذج
repo_name = "fahdsoliman/lfqa_naits"  # اسم النموذج على Huggingface Hub
repo = Repository(local_dir="model_repo", clone_from=repo_name)

set_seed(42)
progress_bar = tqdm(range(num_training_steps), disable=not accelerator.is_local_main_process)
# تدريب النموذج
model.train()
for epoch in range(3):
    train_losses = []
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        train_losses.append(loss.item())
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        progress_bar.set_postfix(loss=round(loss.item(), 3))

    # حساب وعرض متوسط الخسارة لكل دورة تدريبية
    average_train_loss = sum(train_losses) / len(train_losses)
    print(f"Epoch {epoch + 1} Train Loss: {average_train_loss:.4f}")

    # تقييم النموذج على مجموعة التحقق
    model.eval()
    eval_losses = []  # قائمة لتخزين الخسائر لكل دفعة خلال التقييم
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
        eval_losses.append(outputs.loss.item())  # تخزين قيمة الخسارة للدفعة الحالية

    # حساب وعرض متوسط الخسارة لكل دورة تدريبية
    average_eval_loss = sum(eval_losses) / len(eval_losses)
    print(f"Epoch {epoch + 1} Validation Loss: {average_eval_loss:.4f}")

    evaluate_model(preprocessed_dataset2)
    evaluate_model(preprocessed_dataset)
    # العودة لوضع التدريب للمواصلة في الدورة التالية
    model.train()

    # حفظ النموذج في نهاية كل عصر ورفع إلى Huggingface Hub
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained("model_repo", save_function=accelerator.save)
    tokenizer.save_pretrained("model_repo")
    # repo.push_to_hub(commit_message=f"Epoch {epoch + 1} completed")

    print(f"Epoch {epoch + 1} completed.")

# حفظ النموذج النهائي ورفعه إلى Huggingface Hub
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained("model_repo", save_function=accelerator.save)
tokenizer.save_pretrained("model_repo")
# repo.push_to_hub(commit_message="Final model after training")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/fahdsoliman/lfqa_naits into local empty directory.


  0%|          | 0/42 [00:00<?, ?it/s]

Epoch 1 Train Loss: 2.4517
Epoch 1 Validation Loss: 2.3283


<ipython-input-17-593f2c4d81ba>:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_rouge = load_metric("rouge", trust_remote_code=True)


  0%|          | 0/108 [00:00<?, ?it/s]

...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


ROUGE Scores: {'rouge1': 29.429601620762448, 'rouge2': 7.3176895552597845, 'rougeL': 16.518926716937592, 'rougeLsum': 16.524719843662012}
BLEU Score: 1.5070896740039044


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


Epoch 3 completed.


('model_repo/tokenizer_config.json',
 'model_repo/special_tokens_map.json',
 'model_repo/vocab.json',
 'model_repo/merges.txt',
 'model_repo/added_tokens.json',
 'model_repo/tokenizer.json')

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# it all starts with a question/query
# it all starts with a question/query
query = "If I do not pay the service fees within a month and the service is temporarily suspended, can I obtain the information I have on the hosting service?"

# given the question above suppose these documents below were found in some document store
documents = ["Article (45) fees: Hosting service fees are collected according to the fees approved by the Authority. Article (46) fees: The Authority has the right to amend the fees for the hosting service, and the new fees for the service are applied upon the first renewal or request for modification of the current hosting by the customer. Article (47) fees The Authority adopts a pre-payment system for all fees related to the hosting service, Article (48) fees: Public sector entities are excluded from the provisions of Article 44 in this document, as the Authority sends a financial claim after booking the hosting to these entities, provided that the payment notification is received within One month from the date of reservation, Article (49) Fees: The customer can pay location hosting fees every six months, provided that this is mentioned in the location hosting contract.",
             ]



# concatenate question and support documents into BART input
conditioned_doc = "<P> " + " <P> ".join([d for d in documents])
query_and_docs = "question: {} context: {}".format(query, conditioned_doc)

model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)


["If you do not pay the service fees within a month and the service is temporarily suspended, can you obtain the information you have on the hosting service? Yes, you can get the information. If you don't pay any fees within the next month, the service will be temporarily suspended and the information will be lost."]

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# it all starts with a question/query
query = "Why does water heated to room temperature feel colder than the air around it?"

# given the question above suppose these documents below were found in some document store
documents = ["when the skin is completely wet. The body continuously loses water by...",
             "at greater pressures. There is an ambiguity, however, as to the meaning of the terms 'heating' and 'cooling'...",
             "are not in a relation of thermal equilibrium, heat will flow from the hotter to the colder, by whatever pathway...",
             "air condition and moving along a line of constant enthalpy toward a state of higher humidity. A simple example ...",
             "Thermal contact conductance In physics, thermal contact conductance is the study of heat conduction between solid ..."]

# concatenate question and support documents into BART input
conditioned_doc = "<P> " + " <P> ".join([d for d in documents])
query_and_docs = "question: {} context: {}".format(query, conditioned_doc)

model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)



['The air around the water is colder than the air around it. This is due to the difference in temperature between the water and the air, and the difference between the air and the water. The air is warmer than the water, so it feels colder to the water than to the air. The reason the water feels colder is because the water has more surface area to conduct the heat away from it, while the air does not.']

In [ ]:
repo.push_to_hub(commit_message="Final model after training")

Upload file model.safetensors:   0%|          | 1.00/1.51G [00:00<?, ?B/s]

To https://huggingface.co/fahdsoliman/lfqa_naits
   6f68bbc..8adda23  main -> main

   6f68bbc..8adda23  main -> main



'https://huggingface.co/fahdsoliman/lfqa_naits/commit/8adda23499854533f66f8b5f473de9a3b2ee3e0c'